# **Training the UNET**

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import**

In [30]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

**GPU 할당**

In [31]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # 필요할 때 메모리를 할당하도록 설정
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

**Seeding**

In [32]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

**Hyerparameters**

In [33]:
batch_size = 8
lr = 1e-4 #0.0001
epochs = 100
height = 384
width = 256

**Path**

In [34]:
dataset_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug"

files_dir = os.path.join("files", "non-aug")
model_file = os.path.join(r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug", "unet-non-aug.h5")
log_file = os.path.join(files_dir, "log-non-aug.csv")

**Creating Folder**

In [35]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [36]:
create_dir(files_dir)

# **Building UNET**

**Conv Block**

In [37]:
def conv_block(inputs, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters,3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x

**Encoding Block**

In [38]:
def encoder_block(inputs, num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2, 2))(x)
  return x, p

**Decoder Block**

In [39]:
def decoder_block(inputs, skip, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
  x = Concatenate()([x, skip])
  x = conv_block(x, num_filters)
  return x

**UNET**

In [40]:
def build_unet(input_shape):
  inputs = Input(input_shape)

  """Encoder"""
  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  """Bridge"""
  b1 = conv_block(p4, 1024)

  """Decoder"""
  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
  model = Model(inputs, outputs, name="UNET")
  return model

# **Dataset Pipeline**

**Loding the training and validation dataset**

In [41]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
    train_y = sorted(glob(os.path.join(path, "train", "masks", "*.png")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.jpg")))
    test_y = sorted(glob(os.path.join(path, "test", "masks", "*.png")))
    return (train_x, train_y), (test_x, test_y)

**Reading Images**

In [42]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 384)) # 너비와 높이를 줄입니다.
    x = x / 255.0
    return x

**Reading Mask**

In [43]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 384)) # 너비와 높이를 줄입니다.
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    return x


**tf.data pipeline**

In [44]:
def tf_parse(x, y):
  def _parse(x, y):
    x = read_image(x)
    y = read_mask(y)
    return x, y

  x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
  x.set_shape([height, width, 3])
  y.set_shape([height, width, 1])

  return x, y

In [45]:
def tf_dataset(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

**Training**

In [46]:
(train_x, train_y), (test_x, test_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 581 - 581
Test: 101 - 101


In [47]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

In [48]:
for x, y in test_dataset:
  print(x.shape, y.shape)

(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(8, 384, 256, 3) (8, 384, 256, 1)
(5, 384, 256, 3) (5, 384, 256, 1)


In [49]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [50]:

model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 384, 256, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 384, 256, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 384, 256, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 384, 256, 64)         0         ['batch_normalization_18[0]

In [51]:
opt = tf.keras.optimizers.Adam(lr)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [52]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [53]:
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/100
73/73 [==============================] - ETA: 0s - loss: 0.3537 - acc: 0.8315
Epoch 1: val_loss improved from inf to 0.52878, saving model to /content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5
73/73 [==============================] - 66s 770ms/step - loss: 0.3537 - acc: 0.8315 - val_loss: 0.5288 - val_acc: 0.8732 - lr: 1.0000e-04
Epoch 2/100
73/73 [==============================] - ETA: 0s - loss: 0.1958 - acc: 0.8649
Epoch 2: val_loss improved from 0.52878 to 0.36962, saving model to /content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5
73/73 [==============================] - 60s 815ms/step - loss: 0.1958 - acc: 0.8649 - val_loss: 0.3696 - val_acc: 0.8747 - lr: 1.0000e-04
Epoch 3/100
73/73 [==============================] - ETA: 0s - loss: 0.1662 - acc: 0.8673
Epoch 3: val_loss improved from 0.36962 to 0.29872, saving model to /content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-a